In [2]:
from langchain_openai import ChatOpenAI
import pandas as pd
from dotenv import load_dotenv
from datasets import load_dataset, Dataset


from langchain_openai.chat_models import ChatOpenAI
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI

from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

load_dotenv()

/Users/tomasz/Library/Caches/pypoetry/virtualenvs/plan-and-execute-rag-xe948RIs-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")

Repo card metadata block was not found. Setting CardData to empty.


In [16]:

def generate_response_rag_prompt(context, question):
    model = ChatOpenAI(model= "gpt-4o-mini", temperature=0.7)
    prompt = f"""Answer the question: {question}
    Use the provided context to answer the question. The relevant contexts extracted from relevant documents are listed below:
    {context}"""    
    response = model.invoke(prompt)

    return response



# Generate answers for a given model 
ds_for_parse = amnesty_qa["eval"].to_pandas()
results = []
prompts = []
for index, row in ds_for_parse.iterrows(): 
    context = row["contexts"]
    question = row["question"]
    result = generate_response_rag_prompt(context, question)
    results.append(str(result.content))

In [17]:
my_dataset = amnesty_qa["eval"].to_pandas()
my_dataset["answer"] = pd.Series(results)
my_ds_for_eval = Dataset.from_pandas(my_dataset)    


In [18]:
result = evaluate(
    my_ds_for_eval,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)



Evaluating: 100%|██████████| 80/80 [01:01<00:00,  1.30it/s]


In [19]:
result

{'context_precision': 0.9417, 'faithfulness': 0.9244, 'answer_relevancy': 0.9683, 'context_recall': 0.9500}

In [ ]:

# Plan-and-execute response 
def generate_response_plan_and_execute(prompt_input):
    model = ChatOpenAI(model= "gpt-4o-mini", temperature=0.7)
    
    planner = load_chat_planner(model)
    tools = []
    
    executor = load_agent_executor(model, tools, verbose=True)
    
    agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)
    response = agent.invoke({"input": prompt_input})
    return response["output"]

In [ ]:

# Generate answers for plan and execute
ds_for_parse = amnesty_qa["eval"].to_pandas()
results = []
prompts = []
for index, row in ds_for_parse.iterrows(): 
    context = row["contexts"]
    question = row["question"]
    prompt_input = f"""Answer the question: {question}
    Use the provided context to answer the question. The relevant contexts extracted from relevant documents are listed below:
    {context}"""
    prompts.append(prompt_input)
    result = generate_response_plan_and_execute(prompt_input)
    results.append(result)

In [ ]:
my_dataset = amnesty_qa["eval"].to_pandas()
my_dataset["answer"] = pd.Series(results)
my_ds_for_eval = Dataset.from_pandas(my_dataset)

In [ ]:

result = evaluate(
    my_ds_for_eval,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)